Following [this](https://docs.pydantic.dev/usage/types/)

## Overview:

* Where possible pydantic uses *standard library types* to define fields, thus smoothing the learning curve.
* For many useful applications, however, no standard library type exists, so pydantic implements *many commonly used types*.
* If no existing type suits your purpose you can also implement *your own pydantic-compatible types* with custom properties and validation.

## Standard Library Types
* See the full list: https://docs.pydantic.dev/usage/types/#standard-library-types

### Typing Iterables

In [1]:
from typing import Deque, Dict, FrozenSet, List, Optional, Sequence, Set, Tuple, Union

from pydantic import BaseModel

# NOTE where pyright disagrees with pydantic conventions:
# * = None without Optional.
# * Automatic data conversion.


class Model(BaseModel):
    simple_list: list = None
    list_of_ints: List[int] = None

    simple_tuple: tuple = None
    tuple_of_different_types: Tuple[int, float, str, bool] = None

    simple_dict: dict = None
    dict_str_float: Dict[str, float] = None

    simple_set: set = None
    set_bytes: Set[bytes] = None
    frozen_set: FrozenSet[int] = None

    str_or_bytes: Union[str, bytes] = None
    none_or_str: Optional[str] = None

    sequence_of_ints: Sequence[int] = None

    compound: Dict[Union[str, bytes], List[Set[int]]] = None

    deque: Deque[int] = None


print(Model(simple_list=["1", "2", "3"]).simple_list)
# > ['1', '2', '3']
print(Model(list_of_ints=["1", "2", "3"]).list_of_ints)
# > [1, 2, 3]

print(Model(simple_dict={"a": 1, b"b": 2}).simple_dict)
# > {'a': 1, b'b': 2}
print(Model(dict_str_float={"a": 1, b"b": 2}).dict_str_float)
# > {'a': 1.0, 'b': 2.0}

print(Model(simple_tuple=[1, 2, 3, 4]).simple_tuple)
# > (1, 2, 3, 4)
print(Model(tuple_of_different_types=[4, 3, 2, 1]).tuple_of_different_types)
# > (4, 3.0, '2', True)

print(Model(sequence_of_ints=[1, 2, 3, 4]).sequence_of_ints)
# > [1, 2, 3, 4]
print(Model(sequence_of_ints=(1, 2, 3, 4)).sequence_of_ints)
# > (1, 2, 3, 4)

print(Model(deque=[1, 2, 3]).deque)
# > deque([1, 2, 3])

['1', '2', '3']
[1, 2, 3]
{'a': 1, b'b': 2}
{'a': 1.0, 'b': 2.0}
(1, 2, 3, 4)
(4, 3.0, '2', True)
[1, 2, 3, 4]
(1, 2, 3, 4)
deque([1, 2, 3])


### Infinite Generators

**NOTE:**

If you have a generator you can use Sequence as described above.
In that case, **the generator will be consumed and stored on the model** as a list and its values will be validated 
with the sub-type of Sequence (e.g. int in `Sequence[int]`).

But if you have a generator that you don't want to be consumed, e.g. an infinite generator or a remote data loader,
you can define its type with `Iterable`:

In [2]:
from typing import Iterable
from pydantic import BaseModel


class Model(BaseModel):
    infinite: Iterable[int]  # NOTE.


def infinite_ints():  # NOTE.
    i = 0
    while True:
        yield i
        i += 1


m = Model(infinite=infinite_ints())
print(m)
# > infinite=<generator object infinite_ints at 0x103394740>

for i in m.infinite:
    print(i)
    # > 0
    # > 1
    # > 2
    # > 3
    # > 4
    # > 5
    # > 6
    # > 7
    # > 8
    # > 9
    # > 10
    if i == 10:
        break

infinite=<generator object infinite_ints at 0x7f9934240ac0>
0
1
2
3
4
5
6
7
8
9
10


* Iterable fields only perform a simple check that the argument is iterable and won't be consumed.
No validation of their values is performed as it cannot be done without consuming the iterable.

Confusing tip:
> If you want to validate the values of an infinite generator you can create a separate model and use it while consuming
> the generator, reporting the validation errors as appropriate.
>
> pydantic can't validate the values automatically for you because it would require consuming the infinite generator.

#### ℹ️ Validating the first value


In [3]:
import itertools
from typing import Iterable
from pydantic import BaseModel, validator, ValidationError  # NOTE: >> validator <<.
from pydantic.fields import ModelField


class Model(BaseModel):
    infinite: Iterable[int]

    # NOTE this whole pattern - useful:
    @validator("infinite")
    # You don't need to add the "ModelField", but it will help your
    # editor give you completion and catch errors
    def infinite_first_int(cls, iterable, field: ModelField):

        first_value = next(iterable)
        if field.sub_fields:
            # The Iterable had a parameter type, in this case it's int
            # We use it to validate the first value
            sub_field = field.sub_fields[0]
            v, error = sub_field.validate(first_value, {}, loc="first_value")
            if error:
                raise ValidationError([error], cls)  # pyright: ignore

        # This creates a new generator that returns the first value and then
        # the rest of the values from the (already started) iterable
        return itertools.chain([first_value], iterable)


def infinite_ints():
    i = 0
    while True:
        yield i
        i += 1


m = Model(infinite=infinite_ints())
print(m)
# > infinite=<itertools.chain object at 0x102f184c0>


def infinite_strs():
    while True:
        yield from "allthesingleladies"


try:
    Model(infinite=infinite_strs())  # pyright: ignore
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    infinite -> first_value
      value is not a valid integer (type=type_error.integer)
    """

infinite=<itertools.chain object at 0x7f991e7f1580>
1 validation error for Model
infinite -> first_value
  value is not a valid integer (type=type_error.integer)


### Unions

> You may get unexpected coercion with `Union`; see below.
>
> Know that you can also make the check slower but stricter by using Smart Union

In [4]:
from uuid import UUID
from typing import Union
from pydantic import BaseModel


class User(BaseModel):
    id: Union[int, str, UUID]
    name: str


user_01 = User(id=123, name="John Doe")
print(user_01)
# > id=123 name='John Doe'
print(user_01.id)
# > 123
user_02 = User(id="1234", name="John Doe")
print(user_02)
# > id=1234 name='John Doe'
print(user_02.id)
# > 1234
user_03_uuid = UUID(
    "cf57432e-809e-4353-adbd-9d5c0d733868"
)  # NOTE - this is a BIG problem!
user_03 = User(id=user_03_uuid, name="John Doe")
print(user_03)
# > id=275603287559914445491632874575877060712 name='John Doe'
print(user_03.id)
# > 275603287559914445491632874575877060712
print(user_03_uuid.int)
# > 275603287559914445491632874575877060712

id=123 name='John Doe'
123
id=1234 name='John Doe'
1234
id=275603287559914445491632874575877060712 name='John Doe'
275603287559914445491632874575877060712
275603287559914445491632874575877060712


⚠️ However, as can be seen above, pydantic will attempt to 'match' any of the types defined under `Union` and will use **the first one that matches**.

In the above example the id of `user_03` was defined as a `uuid.UUID` class (which is defined under the attribute's 
`Union` annotation) but as the `uuid.UUID` can be marshalled into an `int` it chose to match against the `int` type and **disregarded the other types**!

> ⚠️ As such, it is recommended that, **when defining `Union` annotations, the most specific type is included first and followed by less specific types**.

In [5]:
# Fixed version
from uuid import UUID
from typing import Union
from pydantic import BaseModel


class User(BaseModel):
    id: Union[UUID, int, str]
    name: str


user_03_uuid = UUID("cf57432e-809e-4353-adbd-9d5c0d733868")
user_03 = User(id=user_03_uuid, name="John Doe")
print(user_03)
# > id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
print(user_03.id)
# > cf57432e-809e-4353-adbd-9d5c0d733868
print(user_03_uuid.int)
# > 275603287559914445491632874575877060712

id=UUID('cf57432e-809e-4353-adbd-9d5c0d733868') name='John Doe'
cf57432e-809e-4353-adbd-9d5c0d733868
275603287559914445491632874575877060712


* *Discriminated/Tagged unions* skipped.

* Enums, Datetime, Booleans are all as expected.

* Callable: Callable fields only perform a simple check that the argument is callable; no validation of arguments, their types, or the return type is performed.

### Type

pydantic supports the use of `Type[T]` to specify that a field may only accept classes (not instances) that are subclasses of `T`.

In [7]:
from typing import Type

from pydantic import BaseModel
from pydantic import ValidationError


class Foo:
    pass


class Bar(Foo):
    pass


class Other:
    pass


class SimpleModel(BaseModel):
    just_subclasses: Type[Foo]  # NOTE.


SimpleModel(just_subclasses=Foo)
SimpleModel(just_subclasses=Bar)
try:
    SimpleModel(just_subclasses=Other)
except ValidationError as e:
    print(e)
    """
    1 validation error for SimpleModel
    just_subclasses
      subclass of Foo expected (type=type_error.subclass; expected_class=Foo)
    """

1 validation error for SimpleModel
just_subclasses
  subclass of Foo expected (type=type_error.subclass; expected_class=Foo)


In [8]:
# You may also use Type to specify that any class is allowed.

from typing import Type

from pydantic import BaseModel, ValidationError


class Foo:
    pass


class LenientSimpleModel(BaseModel):
    any_class_goes: Type


LenientSimpleModel(any_class_goes=int)
LenientSimpleModel(any_class_goes=Foo)
try:
    LenientSimpleModel(any_class_goes=Foo())
except ValidationError as e:
    print(e)

    """
    1 validation error for LenientSimpleModel
    any_class_goes
      a class is expected (type=type_error.class)
    """

1 validation error for LenientSimpleModel
any_class_goes
  a class is expected (type=type_error.class)


### TypeVar

`TypeVar` is supported either unconstrained, constrained or with a bound.

In [9]:
from typing import TypeVar
from pydantic import BaseModel

Foobar = TypeVar("Foobar")
BoundFloat = TypeVar("BoundFloat", bound=float)
IntStr = TypeVar("IntStr", int, str)


class Model(BaseModel):
    a: Foobar  # pyright: ignore # equivalent of ": Any"
    b: BoundFloat  # pyright: ignore # equivalent of ": float"
    c: IntStr  # pyright: ignore # equivalent of ": Union[int, str]"


print(Model(a=[1], b=4.2, c="x"))
# > a=[1] b=4.2 c='x'

# a may be None and is therefore optional
print(Model(b=1, c=1))  # pyright: ignore
# > a=None b=1.0 c=1

a=[1] b=4.2 c='x'
a=None b=1.0 c=1


## Literal Type

> This is a new feature of the Python standard library as of Python 3.8;
> prior to Python 3.8, it requires the typing-extensions package.

ℹ️ Useful!

pydantic supports the use of `typing.Literal` (or `typing_extensions.Literal` prior to Python 3.8) as a
lightweight way to specify that a field may accept only specific literal values:

In [10]:
from typing import Literal

from pydantic import BaseModel, ValidationError


class Pie(BaseModel):
    flavor: Literal["apple", "pumpkin"]


Pie(flavor="apple")
Pie(flavor="pumpkin")
try:
    Pie(flavor="cherry")
except ValidationError as e:
    print(str(e))
    """
    1 validation error for Pie
    flavor
      unexpected value; permitted: 'apple', 'pumpkin'
    (type=value_error.const; given=cherry; permitted=('apple', 'pumpkin'))
    """

1 validation error for Pie
flavor
  unexpected value; permitted: 'apple', 'pumpkin' (type=value_error.const; given=cherry; permitted=('apple', 'pumpkin'))


One benefit of this field type is that it can be used to *check for equality with one or more specific values without needing to declare custom validators*:

In [12]:
from typing import ClassVar, List, Union

from typing import Literal

from pydantic import BaseModel, ValidationError


class Cake(BaseModel):
    kind: Literal["cake"]
    required_utensils: ClassVar[List[str]] = ["fork", "knife"]


class IceCream(BaseModel):
    kind: Literal["icecream"]
    required_utensils: ClassVar[List[str]] = ["spoon"]


class Meal(BaseModel):
    dessert: Union[Cake, IceCream]


print(type(Meal(dessert={"kind": "cake"}).dessert).__name__)  # pyright: ignore
# > Cake
print(type(Meal(dessert={"kind": "icecream"}).dessert).__name__)  # pyright: ignore
# > IceCream
try:
    Meal(dessert={"kind": "pie"})
except ValidationError as e:
    print(str(e))
    """
    2 validation errors for Meal
    dessert -> kind
      unexpected value; permitted: 'cake' (type=value_error.const; given=pie;
    permitted=('cake',))
    dessert -> kind
      unexpected value; permitted: 'icecream' (type=value_error.const;
    given=pie; permitted=('icecream',))
    """

Cake
IceCream
2 validation errors for Meal
dessert -> kind
  unexpected value; permitted: 'cake' (type=value_error.const; given=pie; permitted=('cake',))
dessert -> kind
  unexpected value; permitted: 'icecream' (type=value_error.const; given=pie; permitted=('icecream',))


* With proper ordering in an annotated Union, you can use this to parse types of decreasing specificity (see example online)

## Annotated Types

### NamedTuple

In [14]:
from typing import NamedTuple

from pydantic import BaseModel, ValidationError


class Point(NamedTuple):
    x: int
    y: int


class Model(BaseModel):
    p: Point


print(Model(p=("1", "2")))  # pyright: ignore
# > p=Point(x=1, y=2)

try:
    Model(p=("1.3", "2"))
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    p -> x
      value is not a valid integer (type=type_error.integer)
    """

p=Point(x=1, y=2)
1 validation error for Model
p -> x
  value is not a valid integer (type=type_error.integer)


### TypedDict

> This is a new feature of the Python standard library as of Python 3.8. Prior to Python 3.8, it requires the `typing-extensions` package.

In [15]:
from typing_extensions import TypedDict

from pydantic import BaseModel, Extra, ValidationError


# `total=False` means keys are non-required
class UserIdentity(TypedDict, total=False):
    name: str
    surname: str


class User(TypedDict):
    identity: UserIdentity
    age: int


class Model(BaseModel):
    u: User

    class Config:
        extra = Extra.forbid


print(
    Model(u={"identity": {"name": "Smith", "surname": "John"}, "age": "37"})
)  # pyright: ignore
# > u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': 37}

print(
    Model(u={"identity": {"name": None, "surname": "John"}, "age": "37"})
)  # pyright: ignore
# > u={'identity': {'name': None, 'surname': 'John'}, 'age': 37}

print(Model(u={"identity": {}, "age": "37"}))  # pyright: ignore
# > u={'identity': {}, 'age': 37}


try:
    Model(u={"identity": {"name": ["Smith"], "surname": "John"}, "age": "24"})
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    u -> identity -> name
      str type expected (type=type_error.str)
    """

try:
    Model(
        u={
            "identity": {"name": "Smith", "surname": "John"},
            "age": "37",
            "email": "john.smith@me.com",
        }
    )
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    u -> email
      extra fields not permitted (type=value_error.extra)
    """

u={'identity': {'name': 'Smith', 'surname': 'John'}, 'age': 37}
u={'identity': {'name': None, 'surname': 'John'}, 'age': 37}
u={'identity': {}, 'age': 37}
1 validation error for Model
u -> identity -> name
  str type expected (type=type_error.str)
1 validation error for Model
u -> email
  extra fields not permitted (type=value_error.extra)


## Pydantic Types

* See list of pydantic types: https://docs.pydantic.dev/usage/types/#pydantic-types

Broadly:
* URLs and related,
* (HTML) Colors,
* Secrets,
* JSON,
* Payment cards.

Detailed examples skipped.

## Constrained Types

**The value of numerous common types can be restricted using `con*` type functions:**

(For arguments / settings of these `con*` functions see https://docs.pydantic.dev/usage/types/#arguments-to-conlist)

In [16]:
from decimal import Decimal

from pydantic import (
    BaseModel,
    NegativeFloat,
    NegativeInt,
    PositiveFloat,
    PositiveInt,
    NonNegativeFloat,
    NonNegativeInt,
    NonPositiveFloat,
    NonPositiveInt,
    conbytes,
    condecimal,
    confloat,
    conint,
    conlist,
    conset,
    constr,
    Field,
)


# NOTE: con* functions not compatible with pyright at the moment.
class Model(BaseModel):
    upper_bytes: conbytes(to_upper=True)  # pyright: ignore
    lower_bytes: conbytes(to_lower=True)  # pyright: ignore
    short_bytes: conbytes(min_length=2, max_length=10)  # pyright: ignore
    strip_bytes: conbytes(strip_whitespace=True)  # pyright: ignore

    upper_str: constr(to_upper=True)  # pyright: ignore
    lower_str: constr(to_lower=True)  # pyright: ignore
    short_str: constr(min_length=2, max_length=10)  # pyright: ignore
    regex_str: constr(regex=r"^apple (pie|tart|sandwich)$")  # pyright: ignore
    strip_str: constr(strip_whitespace=True)  # pyright: ignore

    big_int: conint(gt=1000, lt=1024)  # pyright: ignore
    mod_int: conint(multiple_of=5)  # pyright: ignore
    pos_int: PositiveInt
    neg_int: NegativeInt
    non_neg_int: NonNegativeInt
    non_pos_int: NonPositiveInt

    big_float: confloat(gt=1000, lt=1024)  # pyright: ignore
    unit_interval: confloat(ge=0, le=1)  # pyright: ignore
    mod_float: confloat(multiple_of=0.5)  # pyright: ignore
    pos_float: PositiveFloat
    neg_float: NegativeFloat
    non_neg_float: NonNegativeFloat
    non_pos_float: NonPositiveFloat

    short_list: conlist(int, min_items=1, max_items=4)  # pyright: ignore
    short_set: conset(int, min_items=1, max_items=4)  # pyright: ignore

    decimal_positive: condecimal(gt=0)  # pyright: ignore
    decimal_negative: condecimal(lt=0)  # pyright: ignore
    decimal_max_digits_and_places: condecimal(
        max_digits=2, decimal_places=2
    )  # pyright: ignore
    mod_decimal: condecimal(multiple_of=Decimal("0.25"))  # pyright: ignore

    bigger_int: int = Field(
        ..., gt=10000
    )  # Look up: https://docs.pydantic.dev/usage/schema/#field-customisation

## Strict Types

You can use the `StrictStr`, `StrictBytes`, `StrictInt`, `StrictFloat`, and `StrictBool` types to prevent coercion from
compatible types. 

These types will only pass validation when the validated value is of the respective type or is a subtype of that type.
This behavior is also exposed via the `strict` field of the `ConstrainedStr`, `ConstrainedBytes`, `ConstrainedFloat` and
`ConstrainedInt` classes and can be combined with a multitude of complex validation rules.

The following caveats apply:

* `StrictBytes` (and the `strict` option of `ConstrainedBytes`) will accept both `bytes`, and `bytearray` types.
* `StrictInt` (and the `strict` option of `ConstrainedInt`) will not accept `bool` types, even though `bool` is a subclass of `int` in Python. Other subclasses will work.
* `StrictFloat` (and the `strict` option of `ConstrainedFloat`) will not accept `int`.


In [17]:
from pydantic import (
    BaseModel,
    StrictBytes,
    StrictBool,
    StrictInt,
    ValidationError,
    confloat,
)


class StrictBytesModel(BaseModel):
    strict_bytes: StrictBytes


try:
    StrictBytesModel(strict_bytes="hello world")
except ValidationError as e:
    print(e)
    """
    1 validation error for StrictBytesModel
    strict_bytes
      byte type expected (type=type_error.bytes)
    """


class StrictIntModel(BaseModel):
    strict_int: StrictInt


try:
    StrictIntModel(strict_int=3.14159)
except ValidationError as e:
    print(e)
    """
    1 validation error for StrictIntModel
    strict_int
      value is not a valid integer (type=type_error.integer)
    """


class ConstrainedFloatModel(BaseModel):
    constrained_float: confloat(strict=True, ge=0.0)  # pyright: ignore


try:
    ConstrainedFloatModel(constrained_float=3)
except ValidationError as e:
    print(e)
    """
    1 validation error for ConstrainedFloatModel
    constrained_float
      value is not a valid float (type=type_error.float)
    """

try:
    ConstrainedFloatModel(constrained_float=-1.23)
except ValidationError as e:
    print(e)
    """
    1 validation error for ConstrainedFloatModel
    constrained_float
      ensure this value is greater than or equal to 0.0
    (type=value_error.number.not_ge; limit_value=0.0)
    """


class StrictBoolModel(BaseModel):
    strict_bool: StrictBool


try:
    StrictBoolModel(strict_bool="False")
except ValidationError as e:
    print(str(e))
    """
    1 validation error for StrictBoolModel
    strict_bool
      value is not a valid boolean (type=value_error.strictbool)
    """

1 validation error for StrictBytesModel
strict_bytes
  byte type expected (type=type_error.bytes)
1 validation error for StrictIntModel
strict_int
  value is not a valid integer (type=type_error.integer)
1 validation error for ConstrainedFloatModel
constrained_float
  value is not a valid float (type=type_error.float)
1 validation error for ConstrainedFloatModel
constrained_float
  ensure this value is greater than or equal to 0.0 (type=value_error.number.not_ge; limit_value=0.0)
1 validation error for StrictBoolModel
strict_bool
  value is not a valid boolean (type=value_error.strictbool)


## ByteSize
* You can use the `ByteSize` data type to convert byte string representation to raw bytes and print out human readable versions of the bytes as well.

## Custom Data Types

> You can also define your own custom data types. There are several ways to achieve it.

⚠️ This is complex.

### Classes with `__get_validators__`

You use a custom class with a classmethod `__get_validators__`.

It will be called to get validators to parse and validate the input data.

> These validators have the same semantics as in [Validators](https://docs.pydantic.dev/usage/validators/),
> you can declare a parameter `config`, `field`, etc.

In [19]:
import re
from pydantic import BaseModel

# https://en.wikipedia.org/wiki/Postcodes_in_the_United_Kingdom#Validation
post_code_regex = re.compile(
    r"(?:"
    r"([A-Z]{1,2}[0-9][A-Z0-9]?|ASCN|STHL|TDCU|BBND|[BFS]IQQ|PCRN|TKCA) ?"
    r"([0-9][A-Z]{2})|"
    r"(BFPO) ?([0-9]{1,4})|"
    r"(KY[0-9]|MSR|VG|AI)[ -]?[0-9]{4}|"
    r"([A-Z]{2}) ?([0-9]{2})|"
    r"(GE) ?(CX)|"
    r"(GIR) ?(0A{2})|"
    r"(SAN) ?(TA1)"
    r")"
)


class PostCode(str):
    """
    Partial UK postcode validation. Note: this is just an example, and is not
    intended for use in production; in particular this does NOT guarantee
    a postcode exists, just that it has a valid format.
    """

    # NOTE explanation here.
    @classmethod
    def __get_validators__(cls):
        # one or more validators may be yielded which will be called in the
        # order to validate the input, each validator will receive as an input
        # the value returned from the previous validator
        yield cls.validate

    # NOTE: Not clear to me what this is for (yet).
    @classmethod
    def __modify_schema__(cls, field_schema):
        # __modify_schema__ should mutate the dict it receives in place,
        # the returned value will be ignored
        field_schema.update(
            # simplified regex here for brevity, see the wikipedia link above
            pattern="^[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}$",
            # some example postcodes
            examples=["SP11 9DG", "w1j7bu"],
        )

    @classmethod
    def validate(cls, v):
        if not isinstance(v, str):
            raise TypeError("string required")
        m = post_code_regex.fullmatch(v.upper())
        if not m:
            raise ValueError("invalid postcode format")
        # you could also return a string here which would mean model.post_code
        # would be a string, pydantic won't care but you could end up with some
        # confusion since the value's type won't match the type annotation
        # exactly
        return cls(f"{m.group(1)} {m.group(2)}")

    def __repr__(self):
        return f"PostCode({super().__repr__()})"


class Model(BaseModel):
    post_code: PostCode


model = Model(post_code="sw8 5el")  # pyright: ignore
print(model)
# > post_code=PostCode('SW8 5EL')
print(model.post_code)
# > SW8 5EL
print(Model.schema())
"""
{
    'title': 'Model',
    'type': 'object',
    'properties': {
        'post_code': {
            'title': 'Post Code',
            'pattern': '^[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}$',
            'examples': ['SP11 9DG', 'w1j7bu'],
            'type': 'string',
        },
    },
    'required': ['post_code'],
}
""";

post_code=PostCode('SW8 5EL')
SW8 5EL
{'title': 'Model', 'type': 'object', 'properties': {'post_code': {'title': 'Post Code', 'pattern': '^[A-Z]{1,2}[0-9][A-Z0-9]? ?[0-9][A-Z]{2}$', 'examples': ['SP11 9DG', 'w1j7bu'], 'type': 'string'}}, 'required': ['post_code']}


### Arbitrary Types Allowed

You can allow arbitrary types using the `arbitrary_types_allowed` config in the [Model Config](https://docs.pydantic.dev/usage/model_config/).

In [20]:
from pydantic import BaseModel, ValidationError


# This is not a pydantic model, it's an arbitrary class
class Pet:
    def __init__(self, name: str):
        self.name = name


class Model(BaseModel):
    pet: Pet
    owner: str

    class Config:
        arbitrary_types_allowed = True


pet = Pet(name="Hedwig")
# A simple check of instance type is used to validate the data
model = Model(owner="Harry", pet=pet)
print(model)
# > pet=<types_arbitrary_allowed.Pet object at 0x10308eef0> owner='Harry'
print(model.pet)
# > <types_arbitrary_allowed.Pet object at 0x10308eef0>
print(model.pet.name)
# > Hedwig
print(type(model.pet))
# > <class 'types_arbitrary_allowed.Pet'>
try:
    # If the value is not an instance of the type, it's invalid
    Model(owner="Harry", pet="Hedwig")
except ValidationError as e:
    print(e)
    """
    1 validation error for Model
    pet
      instance of Pet expected (type=type_error.arbitrary_type;
    expected_arbitrary_type=Pet)
    """

# NOTE:
# Nothing in the instance of the arbitrary type is checked
# Here name probably should have been a str, but it's not validated
pet2 = Pet(name=42)
model2 = Model(owner="Harry", pet=pet2)
print(model2)
# > pet=<types_arbitrary_allowed.Pet object at 0x10308d090> owner='Harry'
print(model2.pet)
# > <types_arbitrary_allowed.Pet object at 0x10308d090>
print(model2.pet.name)
# > 42
print(type(model2.pet))
# > <class 'types_arbitrary_allowed.Pet'>

pet=<__main__.Pet object at 0x7f991e69b610> owner='Harry'
Hedwig
<class '__main__.Pet'>
1 validation error for Model
pet
  instance of Pet expected (type=type_error.arbitrary_type; expected_arbitrary_type=Pet)
pet=<__main__.Pet object at 0x7f991e69ba90> owner='Harry'
42
<class '__main__.Pet'>


In [3]:
# ❗ So by default, arbitrary types are not allowed ❗
# See the error that this will generate below.
# Setting arbitrary_types_allowed *will* check that it is that type still.

from pydantic import BaseModel, ValidationError


# This is not a pydantic model, it's an arbitrary class
class Pet:
    def __init__(self, name: str):
        self.name = name


class Model(BaseModel):
    pet: Pet
    owner: str


pet = Pet(name="Hedwig")

model = Model(owner="Harry", pet=pet)

RuntimeError: no validator found for <class '__main__.Pet'>, see `arbitrary_types_allowed` in Config

### Generic Classes as Types

⚠️ This is very complex.

See: https://docs.pydantic.dev/usage/types/#generic-classes-as-types